In [30]:
! pip install nltk, joblib, pandas, tensorflow

ERROR: Invalid requirement: 'nltk,'


In [31]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import re
import chardet
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import cross_val_score


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
filename = './../datasets/SMS Spam Collection/final/spam.csv'

In [33]:
with open(filename, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
df = pd.read_csv(filename, encoding=result['encoding'])

In [34]:
df.head()

,target,text
0,0,"I am late,so call you tomorrow morning.take ca..."
1,0,U r too much close to my heart. If u go away i...
2,0,Wait &lt;#&gt; min..
3,0,Can you call me plz. Your number shows out of ...
4,0,MAYBE IF YOU WOKE UP BEFORE FUCKING 3 THIS WOU...


In [35]:
# Text Cleaning
df['text'] = df['text'].apply(lambda x: x.lower()) # convert to lower case
df['text'] = df['text'].apply(lambda x: re.sub('[^a-z\s]', '', x)) # remove special characters and numbers

# Tokenization
df['text'] = df['text'].apply(lambda x: word_tokenize(x))

# Stop Word Removal
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Stemming
stemmer = PorterStemmer()
df['text'] = df['text'].apply(lambda x: [stemmer.stem(word) for word in x])

# Convert list of words back to string
df['text'] = df['text'].apply(lambda x: ' '.join(x))

# Vectorization (TF-IDF)
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df['text'])

In [36]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, df['target'], test_size=0.2, random_state=42)

# Defining the models
models = [
    {'label': 'Logistic Regression', 'model': LogisticRegression(C=0.1, random_state=42)},
    {'label': 'Naive Bayes', 'model': MultinomialNB()},
    {'label': 'Random Forest', 'model': RandomForestClassifier(random_state=42)},
    {'label': 'Support Vector Machine', 'model': LinearSVC(C=0.1, random_state=42)},
]

In [37]:
# Training and testing the models
results = []
for m in models:
    model = m['model']
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Implement cross validation
    cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    
    results.append([m['label'], accuracy, precision, recall, f1, cv_score])


In [38]:
# Creating DataFrame with the results
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Cross-Validation Score'])

In [39]:
results_df

,Model,Accuracy,Precision,Recall,F1 Score,Cross-Validation Score
0,Logistic Regression,0.886288,0.991803,0.785714,0.876812,0.906276
1,Naive Bayes,0.933110,0.929487,0.941558,0.935484,0.936402
2,Random Forest,0.936455,0.992701,0.883117,0.934708,0.944770
3,Support Vector Machine,0.923077,0.992481,0.857143,0.919861,0.932218
